In [1]:
import pandas as pd 
import numpy as np
import yfinance
from IPython.display import HTML
import random
#from ipynb.fs.full.Functions1 import hide_toggle
#from operator import itemgetter 
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt
%matplotlib inline
#from mpl_finance import candlestick_ohlc
from mplfinance.original_flavor import candlestick_ohlc
import pandas_datareader as pdr
import datetime as dt
import talib as ta
from itertools import compress
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.signal
import math
from math import sqrt
# import warnings
# warnings.simplefilter("ignore")
import math
from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression

In [41]:
symbol ="GC=F"
num_name= symbol.split(' ') 
df = []
for i in num_name:
        ticker = yfinance.Ticker(i)
        data=ticker.history(interval='60m',period="3mo")
        #data['Date'] = pd.to_datetime(data.index) # just to crate another column for date!!!
        #data['Date'] = data['Date'].apply(mpl_dates.date2num)
        data = data.loc[:,['Open', 'High', 'Low', 'Close']]
#         ohlc_data= data.loc[:,['Date', 'Open', 'High', 'Low', 'Close']].values

        df.append(data)

df[0]


,Open,High,Low,Close
Datetime,,,,
2020-11-19 16:00:00-05:00,1865.400024,1866.699951,1864.400024,1864.500000
2020-11-19 18:00:00-05:00,1864.800049,1864.900024,1859.099976,1861.500000
2020-11-19 19:00:00-05:00,1861.699951,1863.400024,1859.599976,1861.699951
2020-11-19 20:00:00-05:00,1861.699951,1866.500000,1860.900024,1863.900024
2020-11-19 21:00:00-05:00,1863.800049,1864.400024,1861.000000,1863.000000
...,...,...,...,...
2021-02-19 12:00:00-05:00,1782.699951,1784.300049,1777.500000,1782.000000
2021-02-19 13:00:00-05:00,1782.000000,1782.099976,1775.900024,1779.000000
2021-02-19 14:00:00-05:00,1779.199951,1780.000000,1777.699951,1779.099976


In [42]:
Data=[]
for i in range(len(num_name)):
    Data.append(df[i].iloc[:].values)
len(Data)

1

In [43]:
df[0]["Close"][:10] # 7 candles per day!

Datetime
2020-11-19 16:00:00-05:00    1864.500000
2020-11-19 18:00:00-05:00    1861.500000
2020-11-19 19:00:00-05:00    1861.699951
2020-11-19 20:00:00-05:00    1863.900024
2020-11-19 21:00:00-05:00    1863.000000
2020-11-19 22:00:00-05:00    1863.599976
2020-11-19 23:00:00-05:00    1866.900024
2020-11-20 00:00:00-05:00    1867.900024
2020-11-20 01:00:00-05:00    1865.800049
2020-11-20 02:00:00-05:00    1864.300049
Name: Close, dtype: float64

In [44]:
def MacdPlotly(num):
    fig = go.Figure()

    fig = make_subplots(rows=2, cols=1,row_heights=[0.65, 0.35],shared_xaxes=True, vertical_spacing=0.015)
    #Candlestick
    stock_name= num_name[num]
    fig.add_trace(go.Candlestick(x=df[num].index,
                    open=Data[num][:,0],
                    high=Data[num][:,1],
                    low=Data[num][:,2],
                    close=Data[num][:,3], name = f'{stock_name} '),
                 row=1, col=1)
 # Add titles
    fig.update_layout(
        title=f'{stock_name} live share price evolution',
        yaxis_title='Stock Price (USD per Shares)',
        #autosize=True,
        #dragmode='pan'
        height=600, width=1000,
        hovermode='x unified'

        )

    # X-Axes
    fig.update_xaxes(
        rangeslider_visible=False,
        rangebreaks=[
                # NOTE: Below values are bound (not single values), ie. hide x to y
                dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
                dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                # dict(values=["2019-12-25", "2020-12-24"])  # hide holidays (Christmas and New Year's, etc)
            ],
        rangeselector=dict(
            buttons=list([
                dict(count=15, label="15d", step="day", stepmode="backward"),
                dict(count=1, label="1mo", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="todate"),
                dict(count=3, label="3mo", step="month", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    #fig.update_traces(xaxis="x4")
    #fig.update_traces(xaxis='x2') 

    #Show
    fig.show()

In [45]:
for i in range(len(num_name)):
    MacdPlotly(i)

In [46]:
a= df[0]["Close"].shape[0] //30
a

46

In [47]:
df[0]

,Open,High,Low,Close
Datetime,,,,
2020-11-19 16:00:00-05:00,1865.400024,1866.699951,1864.400024,1864.500000
2020-11-19 18:00:00-05:00,1864.800049,1864.900024,1859.099976,1861.500000
2020-11-19 19:00:00-05:00,1861.699951,1863.400024,1859.599976,1861.699951
2020-11-19 20:00:00-05:00,1861.699951,1866.500000,1860.900024,1863.900024
2020-11-19 21:00:00-05:00,1863.800049,1864.400024,1861.000000,1863.000000
...,...,...,...,...
2021-02-19 12:00:00-05:00,1782.699951,1784.300049,1777.500000,1782.000000
2021-02-19 13:00:00-05:00,1782.000000,1782.099976,1775.900024,1779.000000
2021-02-19 14:00:00-05:00,1779.199951,1780.000000,1777.699951,1779.099976


In [48]:
#df[0]["Close"].shape[0]

In [49]:
pts=[]
for i in range(len(num_name)):
    month_diff = df[i]["Close"].shape[0] // (30*7) # Integer divide the number of prices we have by 30 # for HOURLY
    if month_diff == 0: # We want a value greater than 0
        month_diff = 1
    smooth = int(2 * month_diff + 3) # Simple algo to determine smoothness
    pts.append(savgol_filter(df[i]["Close"], smooth, 3)) # Get the smoothened price data
len(pts)

1

In [50]:
for i in range(len(num_name)):
    df[i]["PTS"]= pts[i]


In [51]:
Data=[]
for i in range(len(num_name)):
    Data.append(df[i].iloc[:].values)
len(Data)

1

In [52]:
def MacdPlotly(num):
    fig = go.Figure()

    fig = make_subplots(rows=2, cols=1,row_heights=[0.65, 0.35],shared_xaxes=True, vertical_spacing=0.015)
    #Candlestick
    stock_name= num_name[num]
#     fig.add_trace(go.Candlestick(x=df[num].index,
#                     open=Data[num][:,0],
#                     high=Data[num][:,1],
#                     low=Data[num][:,2],
#                     close=Data[num][:,3], name = f'{stock_name} '),
#                  row=1, col=1)
    
    fig.add_trace(go.Scatter(
            x=df[num].index,
            y=Data[num][:,3],
            name="Normal",
            #fill='tonextx', fillcolor ='violet',
            #marker=dict(color="lightblue"),
            line = dict(color = 'Blue', width=2)
        ),row=1, col=1)
    fig.add_trace(go.Scatter(
            x=df[num].index,
            y=Data[num][:,4],
            name="Smoothened",
            #fill='tonextx', fillcolor ='violet',
            #marker=dict(color="lightblue"),
            line = dict(color = 'Red', width=2)
        ),row=1, col=1)
    
 # Add titles
    fig.update_layout(
        title=f'{stock_name} live share price evolution',
        yaxis_title='Stock Price (USD per Shares)',
        #autosize=True,
        #dragmode='pan'
        height=600, width=1000,
        hovermode='x unified'

        )

    # X-Axes
    fig.update_xaxes(
        rangeslider_visible=False,
        rangebreaks=[
                # NOTE: Below values are bound (not single values), ie. hide x to y
                dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
                dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                # dict(values=["2019-12-25", "2020-12-24"])  # hide holidays (Christmas and New Year's, etc)
            ],
        rangeselector=dict(
            buttons=list([
                dict(count=15, label="15d", step="day", stepmode="backward"),
                dict(count=1, label="1mo", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="todate"),
                dict(count=3, label="3mo", step="month", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    #fig.update_traces(xaxis="x4")
    #fig.update_traces(xaxis='x2') 

    #Show
    fig.show()

In [53]:
for i in range(len(num_name)):
    MacdPlotly(i)

In [54]:
def pythag(pt1, pt2):
    a_sq = (pt2[0] - pt1[0]) ** 2
    b_sq = (pt2[1] - pt1[1]) ** 2
    return sqrt(a_sq + b_sq)

def local_min_max(pts):
    local_min = []
    local_max = []
    prev_pts = [(0, pts[0]), (1, pts[1])]
    for i in range(1, len(pts) - 1):
        append_to = ''
        if pts[i-1] > pts[i] < pts[i+1]:
            append_to = 'min'
        elif pts[i-1] < pts[i] > pts[i+1]:
            append_to = 'max'
        if append_to:
            if local_min or local_max:
                prev_distance = pythag(prev_pts[0], prev_pts[1]) * 0.5
                curr_distance = pythag(prev_pts[1], (i, pts[i]))
                if curr_distance >= prev_distance:
                    prev_pts[0] = prev_pts[1]
                    prev_pts[1] = (i, pts[i])
                    if append_to == 'min':
                        local_min.append((i, pts[i]))
                    else:
                        local_max.append((i, pts[i]))
            else:
                prev_pts[0] = prev_pts[1]
                prev_pts[1] = (i, pts[i])
                if append_to == 'min':
                    local_min.append((i, pts[i]))
                else:
                    local_max.append((i, pts[i]))
    return local_min, local_max

In [55]:
local_min_max1=[]
for i in range(len(num_name)):
    local_min_max1.append(local_min_max(pts[i]))
len(local_min_max1)

1

In [56]:
index_array_for_min=[]
y_values_min=[]
index_array_for_max=[]
y_values_max=[]

for i in range(len(num_name)):
    
    index_array_for_min.append([a[0] for a in local_min_max1[i][0]])
    y_values_min.append([a[1] for a in local_min_max1[i][0]])
    index_array_for_max.append([a[0] for a in local_min_max1[i][1]])
    y_values_max.append([a[1] for a in local_min_max1[i][1]])
    
y_values_min[0]

[1832.4884229841377,
 1801.9837028952306,
 1779.9732076101272,
 1839.061730791335,
 1838.7328552522376,
 1835.625352402089,
 1864.3077134076393,
 1838.2664323211359,
 1836.749219081423,
 1830.6337432170988,
 1859.0991006566492,
 1887.0431795284326,
 1882.5852523596686,
 1877.850300370309,
 1866.6861635234127,
 1878.6286897271034,
 1883.2210048209593,
 1948.2919685467446,
 1915.1923049305467,
 1914.2066903369027,
 1914.2985530525416,
 1836.0919465630407,
 1840.0484901946156,
 1822.2882914133302,
 1864.3864172750036,
 1849.419911822161,
 1852.2647013530577,
 1853.9504761294647,
 1852.284966350583,
 1846.6379986655222,
 1841.0749225599297,
 1834.8232598628497,
 1842.439000194439,
 1851.2448237768767,
 1835.5930238749747,
 1836.3529338854034,
 1821.1056922325824,
 1790.7118782699479,
 1836.6283252836997,
 1842.7324054148507,
 1842.3778421945804,
 1839.1735051081832,
 1823.8368227031,
 1817.855288240607,
 1790.731136877838,
 1773.4381812738718,
 1768.036476266448]

In [57]:
def MacdPlotly(num):
    fig = go.Figure()

    fig = make_subplots(rows=2, cols=1,row_heights=[0.65, 0.35],shared_xaxes=True, vertical_spacing=0.015)
    #Candlestick
    stock_name= num_name[num]
#     fig.add_trace(go.Candlestick(x=df[num].index,
#                     open=Data[num][:,0],
#                     high=Data[num][:,1],
#                     low=Data[num][:,2],
#                     close=Data[num][:,3], name = f'{stock_name} '),
#                  row=1, col=1)
    
    fig.add_trace(go.Scatter(
            x=df[num].index,
            y=Data[num][:,3],
            name="Normal",
            #fill='tonextx', fillcolor ='violet',
            #marker=dict(color="lightblue"),
            line = dict(color = 'Blue', width=2)
        ),row=1, col=1)
    fig.add_trace(go.Scatter(
            x=df[num].index,
            y=Data[num][:,4],
            name="Smoothened",
            #fill='tonextx', fillcolor ='violet',
            #marker=dict(color="lightblue"),
            line = dict(color = 'Red', width=2)
        ),row=1, col=1)
#     fig.add_trace(go.Scatter(    # it is working!
#             x=df[i]["Close"].index[index_array],
#             y=[i[1] for i in local_min_max1[0]],
#             name="min",
#             #fill='tonextx', fillcolor ='violet',
#             mode='markers',
#             marker=dict(
#             size=12,
#             color='black',
#             symbol='cross')
#         ),row=1,col=1)
    fig.add_trace(go.Scatter(   
            x=df[num]["Close"].index[index_array_for_min[num]],
            y=y_values_min[num],
            name="min",
            #fill='tonextx', fillcolor ='violet',
            mode='markers',
            marker=dict(
            size=12,
            color='black',
            symbol='cross')
        ),row=1,col=1)
    fig.add_trace(go.Scatter(   
            x=df[num]["Close"].index[index_array_for_max[num]],
            y=y_values_max[num],
            name="max",
            #fill='tonextx', fillcolor ='violet',
            mode='markers',
            marker=dict(
            size=12,
            color='green',
            symbol='cross')
        ),row=1,col=1)
    
          
               
 # Add titles
    fig.update_layout(
        title=f'{stock_name} live share price evolution',
        yaxis_title='Stock Price (USD per Shares)',
        #autosize=True,
        #dragmode='pan'
        height=600, width=1000,
        hovermode='x unified'

        )

    # X-Axes
    fig.update_xaxes(
        rangeslider_visible=False,
        rangebreaks=[
                # NOTE: Below values are bound (not single values), ie. hide x to y
                dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
                #dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                # dict(values=["2019-12-25", "2020-12-24"])  # hide holidays (Christmas and New Year's, etc)
            ],
        rangeselector=dict(
            buttons=list([
                dict(count=15, label="15d", step="day", stepmode="backward"),
                dict(count=1, label="1mo", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="todate"),
                dict(count=3, label="3mo", step="month", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    #fig.update_traces(xaxis="x4")
    #fig.update_traces(xaxis='x2') 

    #Show
    fig.show()

In [58]:
for i in range(len(num_name)):
    MacdPlotly(i)

In [59]:
df[0].index

DatetimeIndex(['2020-11-19 16:00:00-05:00', '2020-11-19 18:00:00-05:00',
               '2020-11-19 19:00:00-05:00', '2020-11-19 20:00:00-05:00',
               '2020-11-19 21:00:00-05:00', '2020-11-19 22:00:00-05:00',
               '2020-11-19 23:00:00-05:00', '2020-11-20 00:00:00-05:00',
               '2020-11-20 01:00:00-05:00', '2020-11-20 02:00:00-05:00',
               ...
               '2021-02-19 07:00:00-05:00', '2021-02-19 08:00:00-05:00',
               '2021-02-19 09:00:00-05:00', '2021-02-19 10:00:00-05:00',
               '2021-02-19 11:00:00-05:00', '2021-02-19 12:00:00-05:00',
               '2021-02-19 13:00:00-05:00', '2021-02-19 14:00:00-05:00',
               '2021-02-19 15:00:00-05:00', '2021-02-19 15:59:01-05:00'],
              dtype='datetime64[ns, America/New_York]', name='Datetime', length=1380, freq=None)

In [60]:
#For the close values
X = [pt[0] for pt in pts]
y = [pt[1] for pt in pts]

# We need to convert to numpy array, because sklearn expects a numpy array

X = np.array(X)
y = np.array(y)

# We need to reshape X to (-1, 1), because that is the format that sklearn will expect the input array

X = X.reshape(-1, 1)

#Here is how you can do this in a one liner

X = np.array([pt[0] for pt in pts]).reshape(-1, 1)
y = np.array([pt[1] for pt in pts])

In [61]:
series_index= np.arange(df[0]["Close"].shape[0]).reshape(-1, 1)
series_index

array([[   0],
       [   1],
       [   2],
       ...,
       [1377],
       [1378],
       [1379]])

In [62]:
#Data[0][:,4]
a =np.arange(df[0]["Close"].shape[0]).reshape(-1,1) # couldnt do it!


In [63]:

x = a
y = Data[0][:,3]
type(x)

numpy.ndarray

In [64]:
# def regression_ceof(i):
#     x = np.arange(df[i]["Close"].shape[0]).reshape(-1,1)
#     y = Data[i][:,3]
#     model = LinearRegression()
#     model.fit(x, y)
#     return model.coef_[0], model.intercept_

# local_min_slope, local_min_int = regression_ceof(local_min_max1[0][0])
# local_max_slope, local_max_int = regression_ceof(local_min_max1[0][1])

In [65]:
def regression_ceof(pts):
    X = np.array([pt[0] for pt in pts]).reshape(-1, 1)
    y = np.array([pt[1] for pt in pts])
    model = LinearRegression()
    model.fit(X, y)
    return model.coef_[0], model.intercept_
# local_min_slope, local_min_int = regression_ceof(local_min_max1[0][0])
# local_max_slope, local_max_int = regression_ceof(local_min_max1[0][1])

In [66]:
local_min_slope=[] 
local_min_int=[]
local_max_slope=[]
local_max_int=[]
for i in range(len(num_name)):
    local_min_slope.append(regression_ceof(local_min_max1[i][0])[0])
    local_min_int.append(regression_ceof(local_min_max1[i][0])[1])
    local_max_slope.append(regression_ceof(local_min_max1[i][1])[0])
    local_max_int.append(regression_ceof(local_min_max1[i][1])[1])
    
local_max_slope

[-0.0200530274789459]

In [67]:
regression_ceof(local_min_max1[0][0])

(-0.023088697358751558, 1863.2505566520142)

In [68]:
support=[]
resistance=[]
for i in range(len(num_name)):
    series_index= np.arange(df[i]["Close"].shape[0])
    support.append( (local_min_slope[i] * np.array(series_index)) + local_min_int[i])
    resistance.append((local_max_slope[i] * np.array(series_index)) + local_max_int[i])
    
print(len(support),len(resistance))

1 1


In [69]:
def MacdPlotly(num):
    fig = go.Figure()

    fig = make_subplots(rows=2, cols=1,row_heights=[0.65, 0.35],shared_xaxes=True, vertical_spacing=0.015)
    #Candlestick
    stock_name= num_name[num]
#     fig.add_trace(go.Candlestick(x=df[num].index,
#                     open=Data[num][:,0],
#                     high=Data[num][:,1],
#                     low=Data[num][:,2],
#                     close=Data[num][:,3], name = f'{stock_name} '),
#                  row=1, col=1)
    
    fig.add_trace(go.Scatter(
            x=df[num].index,
            y=Data[num][:,3],
            name="Normal",
            #fill='tonextx', fillcolor ='violet',
            #marker=dict(color="lightblue"),
            line = dict(color = 'Blue', width=2)
        ),row=1, col=1)
    fig.add_trace(go.Scatter(
            x=df[num].index,
            y=Data[num][:,4],
            name="Smoothened",
            #fill='tonextx', fillcolor ='violet',
            #marker=dict(color="lightblue"),
            line = dict(color = 'Red', width=2)
        ),row=1, col=1)
#     fig.add_trace(go.Scatter(    # it is working!
#             x=df[i]["Close"].index[index_array],
#             y=[i[1] for i in local_min_max1[0]],
#             name="min",
#             #fill='tonextx', fillcolor ='violet',
#             mode='markers',
#             marker=dict(
#             size=12,
#             color='black',
#             symbol='cross')
#         ),row=1,col=1)
    fig.add_trace(go.Scatter(   
            x=df[num]["Close"].index[index_array_for_min[num]],
            y=y_values_min[num],
            name="min",
            #fill='tonextx', fillcolor ='violet',
            mode='markers',
            marker=dict(
            size=12,
            color='black',
            symbol='cross')
        ),row=1,col=1)
    fig.add_trace(go.Scatter(   
            x=df[num]["Close"].index[index_array_for_max[num]],
            y=y_values_max[num],
            name="max",
            #fill='tonextx', fillcolor ='violet',
            mode='markers',
            marker=dict(
            size=12,
            color='green',
            symbol='cross')
        ),row=1,col=1)
    
#Support Resistance
    fig.add_trace(go.Scatter(   
            x=df[num].index,
            y=support[num],
            name="support",
            line = dict(color = 'Blue', width=2)
            #fill='tonextx', fillcolor ='violet',
            #mode='markers',
#             marker=dict(
#             size=12,
#             color='red',
#             symbol='cross')
        ),row=1,col=1)
    fig.add_trace(go.Scatter(   
            x=df[num].index,
            y=resistance[num],
            name="resistance",
            line = dict(color = 'black', width=2)
            #fill='tonextx', fillcolor ='violet',
            #mode='markers',
#             marker=dict(
#             size=12,
#             color='red',
#             symbol='cross')
        ),row=1,col=1)
    
    
          
               
 # Add titles
    fig.update_layout(
        title=f'{stock_name} live share price evolution',
        yaxis_title='Stock Price (USD per Shares)',
        #autosize=True,
        #dragmode='pan'
        height=600, width=1000,
        hovermode='x unified'

        )

    # X-Axes
    fig.update_xaxes(
        rangeslider_visible=False,
        rangebreaks=[
                # NOTE: Below values are bound (not single values), ie. hide x to y
                dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
                #dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                # dict(values=["2019-12-25", "2020-12-24"])  # hide holidays (Christmas and New Year's, etc)
            ],
        rangeselector=dict(
            buttons=list([
                dict(count=15, label="15d", step="day", stepmode="backward"),
                dict(count=1, label="1mo", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="todate"),
                dict(count=3, label="3mo", step="month", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    #fig.update_traces(xaxis="x4")
    #fig.update_traces(xaxis='x2') 

    #Show
    fig.show()

In [70]:
for i in range(len(num_name)):
    MacdPlotly(i)